<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/qiskit_kernel_sweep_extended_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit scikit-learn numpy matplotlib

In [ ]:
#!/usr/bin/env python3
"""
qiskit_kernel_sweep_extended.py

Extended fidelity‐kernel sweep for multiple feature maps, true imbalance,
and weighted vs. unweighted SVM across depths.

- Generates toy 2D data with various imbalance ratios.
- Builds ZZ and Pauli feature maps using Qiskit’s API.
- Computes fidelity^2 Gram matrices, centers them, and measures alignment.
- Runs StratifiedKFold CV with SVC(precomputed), with/without class weights.
"""

import warnings
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from qiskit.circuit.library import zz_feature_map, PauliFeatureMap
from qiskit.quantum_info import Statevector, state_fidelity

# Suppress Qiskit deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def generate_data(n_samples, imbalance, random_state=None):
    """Create 2D classification data with specified class imbalance."""
    if imbalance >= 1.0:
        weights = [0.5, 0.5]
    else:
        weights = [imbalance, 1.0 - imbalance]
    X, y = make_classification(n_samples=n_samples,
                               n_features=2,
                               n_redundant=0,
                               n_clusters_per_class=1,
                               weights=weights,
                               class_sep=2.0,
                               random_state=random_state)
    return X, y

def build_feature_map(name, depth):
    """Return a QuantumCircuit feature map by name and repetition depth."""
    if name == "ZZ":
        return zz_feature_map(feature_dimension=2,
                              reps=depth,
                              entanglement='circular',
                              insert_barriers=False)
    elif name == "Pauli":
        return PauliFeatureMap(feature_dimension=2,
                               reps=depth,
                               paulis=['X','Y','Z'],
                               entanglement='circular',
                               insert_barriers=False)
    else:
        raise ValueError(f"Unknown feature map: {name}")

def compute_fidelity_kernel(fmap, X):
    """Compute fidelity^2 Gram matrix for all pairs in X."""
    n = len(X)
    # prepare statevectors
    psi = [Statevector.from_instruction(fmap.assign_parameters(x)) for x in X]
    K = np.zeros((n, n), float)
    for i in range(n):
        for j in range(i, n):
            f2 = state_fidelity(psi[i], psi[j])**2
            K[i, j] = K[j, i] = f2
    return K

def center_kernel(K):
    """Center a kernel matrix in feature space."""
    n = K.shape[0]
    one_n = np.ones((n, n)) / n
    return K - one_n @ K - K @ one_n + one_n @ K @ one_n

def kernel_target_alignment(Kc, y):
    """Compute the kernel-target alignment metric."""
    yy = np.outer(y, y)
    num = np.trace(Kc @ yy)
    den = np.linalg.norm(Kc, 'fro') * np.linalg.norm(yy, 'fro')
    return num / den

def svm_cv(K, y, weighted=False, max_splits=5):
    """Return CV scores for SVC with precomputed kernel."""
    classes, counts = np.unique(y, return_counts=True)
    n_splits = min(max_splits, counts.min())
    if n_splits < 2:
        return np.array([])
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    svc = SVC(kernel='precomputed',
              class_weight='balanced' if weighted else None)
    return cross_val_score(svc, K, y, cv=cv)

def main():
    np.random.seed(42)
    n_samples = 50
    imbalance_levels = [1.0, 0.75, 0.5, 0.2, 0.1]  # include true skew at 20% & 10%
    feature_maps = ["ZZ", "Pauli"]
    depths = [1, 3, 5]

    for imb in imbalance_levels:
        X, y = generate_data(n_samples, imb, random_state=1)
        header = f"Imbalance {int(imb*100)}%: N={n_samples}"
        print(header)
        print("-" * len(header))

        for name in feature_maps:
            for d in depths:
                fmap = build_feature_map(name, d)
                K = compute_fidelity_kernel(fmap, X)
                Kc = center_kernel(K)
                align = kernel_target_alignment(Kc, y)

                scores = svm_cv(K, y, weighted=False)
                scores_w = svm_cv(K, y, weighted=True)

                def fmt(s): return f"{s.mean():.3f}±{s.std():.3f}" if s.size > 0 else "n/a"
                print(f"{name:6s} depth={d} | align={align:.3f} "
                      f"| SVM={fmt(scores)} | SVM_bal={fmt(scores_w)}")
        print()

if __name__ == "__main__":
    main()